In [60]:
# 导入python包，并配置部分定量
import os
import glob
from moviepy.editor import VideoFileClip
import subprocess
import shutil
from IPython.display import display, Image
WORKING_DIR = r'F:\VideosCache\MakingDirs\InputVideos'
OUTPUT_DIR = r'F:\VideosCache\MakingDirs\OutputVideos'
MAINVIDEOS_DIR = os.path.join(WORKING_DIR,'Mainvideos')
PICTURERESOURCE_DIR = os.path.join(WORKING_DIR,'PictureResource')
RUBBISHCACHE_DIR = os.path.join(WORKING_DIR,'RubbishCache')
REINDEX_JPG_DIR = os.path.join(RUBBISHCACHE_DIR,'ReIndex_Jpg')   # 重排后的文件存储在这里

TARGET_FRAME_NUM = 30 # 目标帧数

In [2]:
# 获取目录文件夹下所有的视频文件
def get_all_videos(file_dir):
    videos_files = []
    videos_suffixs = ['avi','MP4','flv','mov','mp4']
    for suffix in videos_suffixs:
        file = glob.glob(os.path.join(file_dir,'*.%s'%suffix))
        if len(file):
            videos_files.extend(file)
    return videos_files
Main_videos = get_all_videos(MAINVIDEOS_DIR) # 获取main videos
Picture_resource_videos = get_all_videos(PICTURERESOURCE_DIR) # 获取画中画资源

In [3]:
# Now begin to deal one video
# Are you ready?
# 打开视频文件
dealing_video = Main_videos[0]
clip = VideoFileClip(dealing_video)

In [19]:
# 截取 前0.1s和后0.1s
def subclip_0_1(clip_):
    print('截取0.1s前的时长:{:.2f}'.format(clip_.duration))
    clip_ = clip_.subclip(0.1,-0.1)
    print('截取0.1s后的时长:{:.2f}'.format(clip_.duration))

In [62]:
# 脚本文件cell
# 清空文件夹
def clean_dir(dir_path):
    files = glob.glob(os.path.join(dir_path,'*'))
    for file in files:
        if not os.path.isdir(file):
            os.remove(file)
        else:
            clean_dir(file)

def regenerate_index(dir_path=RUBBISHCACHE_DIR):
    if not os.path.exists(REINDEX_JPG_DIR):
        os.makedirs(REINDEX_JPG_DIR)
    jpg_png = glob.glob(os.path.join(dir_path,'*.jpg'))
    path = sorted(jpg_png, key=lambda name: float(name.split('\\')[-1][:-4]))
    for index, file in enumerate(path):
        refine_index = index + 1   # 从1开始计算
        shutil.copy(file, os.path.join(REINDEX_JPG_DIR,'%05d.jpg'%refine_index))  # 可以复制到目录下


In [61]:

# use ffmpeg to convert videos to jpgs....
def videos_to_jpgs(clip_,frame_number=30):
    videos_file = clip_.filename
    cmd = 'ffmpeg -i {} -r {} -f image2 {}\%05d.jpg'.format(videos_file,frame_number,RUBBISHCACHE_DIR)
    print(cmd)
    # result=subprocess.Popen(cmd,shell=False,stdout=subprocess.PIPE).stdout


# 删除部分帧数
def delete_some_frame(frame_number=30):
    jpgs_files_ = os.path.join(RUBBISHCACHE_DIR,'*.jpg') # jgps的glob文件匹配名
    jpgs_files = glob.glob(jpgs_files_)
    print('视频提取的图片总数:{}'.format(len(jpgs_files)))
    delete_num = len(jpgs_files) // frame_number # 需要删除的图片数量
    delete_pics_index = [29*i for i in range(1,delete_num+1)]  # 需要删除图片的indexs
    for index in delete_pics_index:
        os.remove(jpgs_files[index])
    print('succeed to delete the setting frame.')
    print('视频抽除之后的图片总数:{}'.format(len(glob.glob(jpgs_files_))))

# 图片帧拼接为视频
def frame_splicing(jpgs_dir=REINDEX_JPG_DIR,target_video_name='output.mp4',frame_number=30):
    target_video = os.path.join(OUTPUT_DIR,target_video_name)
    jpgs_file = os.path.join(jpgs_dir,'%05d.jpg')
    # command = 'ffmpeg -i ./data/cat-girl/%05d.jpg -vcodec mpeg4 -r 30 testavi.avi'
    command = 'ffmpeg -i {} -r {} {}'.format(jpgs_file,frame_number,target_video)
    command_vcodec = 'ffmpeg -i {} -vcodec mpeg4 -r {} {}'.format(jpgs_file,frame_number,target_video)
    print('command_vcodec:{}'.format(command))
    subprocess.run(command_vcodec)


def frame_extraction(clip_,frame_number=30):
    print('视频的帧数为{:.2f}'.format(clip_.fps))
    # clean_dir(RUBBISHCACHE_DIR)
    # videos_to_jpgs(clip_)    # 将视频抽帧为jpgs
    # delete_some_frame()      # 删除部分帧数
    frame_splicing()         # 实现帧拼接
frame_extraction(clip)


视频的帧数为30.00
command_vcodec:ffmpeg -i F:\VideosCache\MakingDirs\InputVideos\RubbishCache\ReIndex_Jpg\%05d.jpg -r 30 F:\VideosCache\MakingDirs\OutputVideos\output.mp4


In [63]:
# 重新对图片进行排序
regenerate_index()

In [56]:
# 清空目标文件夹
clean_dir(RUBBISHCACHE_DIR)

In [5]:
# //mp4左右镜像翻转
# ffmpeg -i input.mp4 -vf "hflip" outut.mp4

#//mp4上下镜像翻转
# ffmpeg -i input.mp4 -vf "vflip" outut.mp4